<a href="https://colab.research.google.com/github/CodeByAtharva/Agentic-AI-Internship/blob/master/day%207/day_7_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_google_genai requests -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from datetime import datetime
from langchain_core.tools import tool

In [10]:
!pip install langchain langchain-community langchain-openai duckduckgo-search
!pip install langchain
!pip install langchain-google-genai
!pip install google-generativeai
!pip install tavily-python
!pip install -U ddgs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 14.0 MB/s eta 0:00:00


In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableBranch



gemini_key = userdata.get("GeminiApi")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    api_key = gemini_key
)





intent_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
You are a news assistant.

From the user query below, extract:
1. topic (short phrase)
2. type: one of [headlines, deep_dive, mixed]

Return in exactly this format:
topic: <topic>
type: <type>

User query: {query}
"""
)

intent_chain = (
    intent_prompt
    | llm
    | StrOutputParser()
)

def parse_intent(text: str):
    lines = text.strip().splitlines()
    topic = lines[0].replace("topic:", "").strip()
    info_type = lines[1].replace("type:", "").strip()
    return {"topic": topic, "type": info_type}


search_tool = DuckDuckGoSearchResults(output_format="list")

def search_news(inputs):
    results = search_tool.invoke(inputs["topic"])
    return {
        **inputs,
        "results": results[:5]   # top 5 results
    }


def format_results(inputs):
    formatted = []
    for i, r in enumerate(inputs["results"], 1):
        formatted.append(
            f"Headline {i}: {r.get('title')}\n"
            f"Description: {r.get('snippet')}\n"
            f"Link: {r.get('link')}\n"
        )
    return {
        **inputs,
        "context": "\n".join(formatted)
    }


headlines_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are a news assistant.

Given the following news articles, generate 3–5 short bullet point headlines.
Each bullet should be one sentence.

Articles:
{context}
"""
)

deep_dive_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are a news research assistant.

Write a clear, beginner-friendly summary (2–4 short paragraphs) explaining:
- What is happening
- Why it matters
- Key names, dates, or numbers

Articles:
{context}
"""
)

mixed_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are a news assistant.

First, provide 3 short bullet point headlines.
Then, write a brief summary (1–2 paragraphs).

Articles:
{context}
"""
)

headlines_chain = headlines_prompt | llm | StrOutputParser()
deep_dive_chain = deep_dive_prompt | llm | StrOutputParser()
mixed_chain = mixed_prompt | llm | StrOutputParser()

router = RunnableBranch(
    (lambda x: x["type"] == "headlines", headlines_chain),
    (lambda x: x["type"] == "deep_dive", deep_dive_chain),
    (lambda x: x["type"] == "mixed", mixed_chain),
    mixed_chain
)

news_assistant = (
    {"query": RunnablePassthrough()}
    | intent_chain
    | RunnableLambda(parse_intent)
    | RunnableLambda(search_news)
    | RunnableLambda(format_results)
    | router
)



user_query = input("Enter the topic: ")
response = news_assistant.invoke(user_query)
print(response)


Enter the topic: What are today’s top tech news headlines?
Here are 3-5 short bullet point headlines based on the provided articles:

*   CNN offers breaking news and daily updates on U.S. and world events.
*   Google News aggregates comprehensive, up-to-date news coverage from various sources.
*   TechCrunch provides daily news and industry insights for startups and technology.
*   Yahoo News Singapore delivers the latest technology updates and headlines globally.
